In [16]:
from specparam import SpectralGroupModel
from specparam.plts.spectra import plot_spectra
import mne
from spectral.utils import print_date_time, read_parameters, config_project
from pathlib import Path
import re
from spectral.specparam import specparam2pandas
import pandas as pd


def extract_numbers(bids_paths):
    # Initialize an empty list to store the numbers
    numbers = []

    # Get all the child directories
    for path in bids_paths.glob("*"):
        if path.is_dir():
            # Convert the path to a string
            str_path = str(path)

            # Find all three-digit numbers in the string
            matches = re.findall(r"\b\d{3}\b", str_path)

            # Add the numbers to the list
            numbers.extend(matches)

    # Now 'numbers' is a list of all three-digit numbers in the child directory names
    return sorted(numbers)


fg = SpectralGroupModel(
    peak_width_limits=[1, 6],
    min_peak_height=0.05,
    peak_threshold=2.0,
    max_n_peaks=6,
    verbose=False,
)

project_path = "/Users/daniel/Projects/psd-path"
specparam_path = f"{project_path}/analysis/data/specparam/2024-05-14_specparam"
bids_folder = Path(f"{project_path}/data/raw/BIDS/")
subjects = extract_numbers(bids_folder)
for subject in subjects:
    print(f"Processing subject {subject}")
    my_paths = config_project(subject)
    project_path = my_paths["project_path"]
    epochs_path = f"{project_path}/analysis/sub-{subject}_interpolated-epo.fif"
    epochs_ar = mne.read_epochs(epochs_path)
    # Print the number of channels
    print(f"Number of channels: {len(epochs_ar.info['ch_names'])}")

    # Print the bad channels
    print(f"Bad channels: {epochs_ar.info['bads']}")
    psd = epochs_ar.compute_psd().average()
    spectra, freqs = psd.get_data(return_freqs=True)
    # Initialize a FOOOFGroup object, with desired settings

    # Define the frequency range to fit
    freq_range = [2, 40]
    fg.fit(freqs, spectra, freq_range)
    channel_names = epochs_ar.info["ch_names"]
    df_channels = pd.DataFrame({"ID": range(len(channel_names)), "ch": channel_names})

    df = specparam2pandas(fg)
    df = df.merge(df_channels, on="ID")
    df["sub_id"] = subject
    # Define the new column order
    cols = ["sub_id", "ch"] + [col for col in df.columns if col not in ["sub_id", "ch"]]
    # Reindex the DataFrame
    df = df.reindex(columns=cols)
    df.to_csv(f"{specparam_path}/sub-{subject}-specparam.csv", index=False)
    print(f"Subject {subject} done")

Processing subject 101
Reading /Users/daniel/PhD/Projects/psd-path/analysis/sub-101_interpolated-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
76 matching events found
No baseline correction applied
0 projection items activated
Number of channels: 205
Bad channels: []
    Using multitaper spectrum estimation with 7 DPSS windows
Subject 101 done
Processing subject 102
Reading /Users/daniel/PhD/Projects/psd-path/analysis/sub-102_interpolated-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    5000.00 ms
        0 CTF compensation matrices available
Not setting metadata
74 matching events found
No baseline correction applied
0 projection items activated
Number of channels: 205
Bad channels: []
    Using multitaper spectrum estimation with 7 DPSS windows
Subject 102 done
Processing subject 103
Reading /Users/daniel/PhD/Projects/psd-path/analysis/sub-103_interpolat